In [1]:
# Importing libraries
from bs4 import BeautifulSoup
import pandas as pd
import requests

## Q1. Getting all the headers from the wiipedia page

In [3]:

def fnReturnHeaders_GivenURL(sInput_Url):
    wikipedia_page=requests.get(sInput_Url)
    wikipedia_soup=BeautifulSoup(wikipedia_page.content,'html.parser')
    h1_headers=wikipedia_soup.find_all('h1')
    h2_headers=wikipedia_soup.find_all('h2')
    h3_headers=wikipedia_soup.find_all('h3')
    outputList=h1_headers+h2_headers+h3_headers
    return(outputList)

In [4]:
Wikipedia_URL="http://en.wikipedia.org/wiki/Main_Page"
sHeadersList=fnReturnHeaders_GivenURL(Wikipedia_URL)
print(sHeadersList)
# As we can see all the headers are printed

[<h1 class="firstHeading" id="firstHeading" lang="en">Main Page</h1>, <h2 class="mp-h2" id="mp-tfa-h2"><span id="From_today.27s_featured_article"></span><span class="mw-headline" id="From_today's_featured_article">From today's featured article</span></h2>, <h2 class="mp-h2" id="mp-dyk-h2"><span class="mw-headline" id="Did_you_know_...">Did you know ...</span></h2>, <h2 class="mp-h2" id="mp-itn-h2"><span class="mw-headline" id="In_the_news">In the news</span></h2>, <h2 class="mp-h2" id="mp-otd-h2"><span class="mw-headline" id="On_this_day">On this day</span></h2>, <h2 class="mp-h2" id="mp-tfp-h2"><span id="Today.27s_featured_picture"></span><span class="mw-headline" id="Today's_featured_picture">Today's featured picture</span></h2>, <h2 class="mp-h2" id="mp-other"><span class="mw-headline" id="Other_areas_of_Wikipedia">Other areas of Wikipedia</span></h2>, <h2 class="mp-h2" id="mp-sister"><span id="Wikipedia.27s_sister_projects"></span><span class="mw-headline" id="Wikipedia's_sister_pr

## Q2. Get imdb's top rated 100 movies and store it in csv file

In [6]:
# Function to get name and year of highest rated movies
def fnReturnList_OfMovieNameAndYear(movieYear_List):
    lIndex_dot=0
    lIndex_op=0 # op: Open paranthesis
    lIndex_cp=0 # cp: closed paranthesis
    MovieName=[]
    Year=[]
    for sText in movieYear_List:
        # extract text from html tags and replace \n with empty string
        sText=sText.get_text().replace("\n","")
        # Get indices
        lIndex_dot=sText.index('.')
        lIndex_op=sText.index('(')
        lIndex_cp=sText.index(')')
        MovieName.append(sText[lIndex_dot+1:lIndex_op])
        Year.append(sText[lIndex_op+1:lIndex_cp])
    return(MovieName,Year)


In [7]:
# This function returns the dataframe of top rated 100 movies
def fnReturnTop_RatedHundredMovies_AsDataFrame(sURL,moviename_tag,class_moviename,ratings_tag,class_ratings):
    Output_df=pd.DataFrame()
    Movie_Name=[]
    Year=[]
    ratings_list=[]
    IMDB_page=requests.get(sURL)
    IMDB_Data_Soup=BeautifulSoup(IMDB_page.content,'html.parser')
    MovieNames=IMDB_Data_Soup.find_all(moviename_tag,class_=class_moviename)
    Movie_Name,Year=fnReturnList_OfMovieNameAndYear(MovieNames)
    ratings=IMDB_Data_Soup.find_all(ratings_tag,class_=class_ratings)
    for sText in ratings:
        ratings_list.append(sText.get_text().replace("\n",""))
    Output_df['Movie_Name']=Movie_Name
    Output_df['Year']=Year
    Output_df['Ratings']=ratings_list
    return(Output_df)

In [8]:
sImdb_URL="https://www.imdb.com/search/title/?count=100&groups=top_1000&sort=user_rating"
output_df=fnReturnTop_RatedHundredMovies_AsDataFrame(sImdb_URL,'h3',"lister-item-header",'div',"inline-block ratings-imdb-rating")

In [9]:
# Storing data frame in a csv file
output_df.to_csv('TopRatedMovies.csv')

## Q3. Top 100 Bollywood movies

In [10]:
sImdb_URL="https://www.imdb.com/india/top-rated-indian-movies/"
Output_df=fnReturnTop_RatedHundredMovies_AsDataFrame(sImdb_URL,'td',"titleColumn",'td',"ratingColumn imdbRating")

In [11]:
# Storing first 100 rows
Output_df=Output_df.loc[0:99]

output_df.to_csv('TopRatedBollywoodMovies.csv')

## Q4. Getting book review of top 5 books

In [12]:
# Function returns the data frame of book details of input url
def fnReturnDataFrame_ContainingBookDetails(sURL):
    Output_df=pd.DataFrame()
    Book_Names=[]
    Authors=[]
    Genre=[]
    Reviews=[]
    Book_page=requests.get(sURL)
    Book_Data_Soup=BeautifulSoup(Book_page.content,'html.parser')
    BookNames=Book_Data_Soup.find_all('h4',class_="italic")
    Author_Names=Book_Data_Soup.find_all('p',class_="sans bold")
    genre=Book_Data_Soup.find_all('p',class_="genre-links hidden-phone")
    BR=Book_Data_Soup.find_all('p',class_="excerpt")
    for lIndex in range(5):
        Book_Names.append(BookNames[lIndex].get_text().replace("\n",""))
        Authors.append(Author_Names[lIndex].get_text().replace("\n",""))
        Genre.append(genre[lIndex].get_text().replace("\n",""))
        Reviews.append(BR[lIndex].get_text().replace("\n",""))
        
    Output_df['Book_Names']=Book_Names
    Output_df['Authors']=Authors
    Output_df['Genres']=Genre
    Output_df['Reviews']=Reviews
    return(Output_df)

In [13]:
sURL="https://bookpage.com/reviews"
# Calling the function and storing the book details
Books_df=fnReturnDataFrame_ContainingBookDetails(sURL)

## Q5. Getting ICC cricket rankings for Men ODI cricket

In [74]:
def fnGetTop10_Cricket_TeamDetails(sURL,sTeam_tag,sTeam_class,
                            sFirstTeamMatch_tag,sFirstTeamMatch_class,sMatch_tag,sMatch_class,
                            sFirstTeamPoint_tag,sFirstTeamPoint_class,sPoint_tag,sPoint_class,
                            sFirstTeamRating_tag,sFirstTeamRating_class,sRating_tag,sRating_class):
    # DataFrame Initialization
    Output_df=pd.DataFrame()
    
    #List initializations
    Teams_list=[]
    Match_list=[]
    Points_list=[]
    Ratings_list=[]
    
    #Requesting webpage for information from input URL
    Cricket_page=requests.get(sURL)
    
    #Reading the content
    Cricket_soup=BeautifulSoup(Cricket_page.content,'html.parser')
    
    # Getting information for the team at first rank and storing them in respective lists
    Teams=Cricket_soup.find_all(sTeam_tag,class_=sTeam_class)
    TeamMatches=Cricket_soup.find_all(sFirstTeamMatch_tag,class_=sFirstTeamMatch_class)
    TeamPoints=Cricket_soup.find_all(sFirstTeamPoint_tag,class_=sFirstTeamPoint_class)
    TeamRatings=Cricket_soup.find_all(sFirstTeamRating_tag,class_=sFirstTeamRating_class)
    
    Teams_list.append(Teams[0].get_text().replace("\n",""))
    Match_list.append(TeamMatches[0].get_text().replace("\n",""))
    Points_list.append(TeamPoints[0].get_text().replace("\n",""))
    sText=TeamRatings[0].get_text().replace("\n","")
    sText=sText.strip()
    Ratings_list.append(sText)
    
    # Getting information for the team starting from rank 2 and storing them in respective lists
    TeamMatches=Cricket_soup.find_all(sMatch_tag,class_=sMatch_class)
    TeamPoints=Cricket_soup.find_all(sPoint_tag,class_=sPoint_class)
    TeamRatings=Cricket_soup.find_all(sRating_tag,class_=sRating_class)
    
    for lIndex in range(9):
        Teams_list.append(Teams[lIndex+1].get_text().replace("\n",""))
    #   Match_list.append(TeamMatches[lIndex].get_text().replace("\n",""))
    #   Points_list.append(TeamMatches[lIndex+1].get_text().replace("\n",""))
        Ratings_list.append(TeamRatings[lIndex].get_text().replace("\n",""))
    
    for lIndex in range(0,18,2):
        Match_list.append(TeamMatches[lIndex].get_text().replace("\n",""))
        Points_list.append(TeamMatches[lIndex+1].get_text().replace("\n",""))
    
    # Storing lists as columns/features in output dataframe
    Output_df['Teams']=Teams_list
    Output_df['Total_Matches']=Match_list
    Output_df['Points']=Points_list
    Output_df['Ratings']=Ratings_list
    
    return(Output_df)

In [49]:
# Getting top 10 ODI teams in men cricket
sURL="https://www.icc-cricket.com/rankings/mens/team-rankings/odi"
TopODITeams_df=fnGetTop10_Cricket_TeamDetails(sURL,'span',"u-hide-phablet",'td',
                "rankings-block__banner--matches",'td',"table-body__cell u-center-text",'td',
                "rankings-block__banner--points",'td',"table-body__cell u-center-text",'td',
                "rankings-block__banner--rating u-text-right",'td',"table-body__cell u-text-right rating")
TopODITeams_df.head()

['England', 'India', 'New Zealand', 'Australia', 'South Africa', 'Pakistan', 'Bangladesh', 'Sri Lanka', 'West Indies', 'Afghanistan']
['44', '52', '32', '39', '31', '35', '34', '39', '43', '28']
['5,405', '6,102', '3,716', '4,344', '3,345', '3,490', '2,989', '3,297', '3,285', '1,549']
['123', '117', '116', '111', '108', '100', '88', '85', '76', '55']


Teams Total_Matches Points Ratings
0       England            44  5,405     123
1         India            52  6,102     117
2   New Zealand            32  3,716     116
3     Australia            39  4,344     111
4  South Africa            31  3,345     108

In [92]:
def fnGetTop10_ODIBatsmen_Details(sURL):
    # DataFrame Initialization
    Output_df=pd.DataFrame()
    
    #List initializations
    Names_list=[]
    Teams_list=[]
    Ratings_list=[]
    
    # Requesting webpage for information from input URL
    Cricket_page=requests.get(sURL)
    
    # Reading the content
    Cricket_soup=BeautifulSoup(Cricket_page.content,'html.parser')
    
    # Getting information for the batsmen at first rank and storing them in respective lists
    Names=Cricket_soup.find_all('div',class_="rankings-block__banner--name")
    Teams=Cricket_soup.find_all('div',class_="rankings-block__banner--nationality")
    Ratings=Cricket_soup.find_all('div',class_="rankings-block__banner--rating")
    
    Names_list.append(Names[0].get_text().replace("\n",""))
    Teams_list.append(Teams[0].get_text().replace("\n",""))
    Ratings_list.append(Ratings[0].get_text().replace("\n",""))
    
    # Getting list of other 9 players
    Names=Cricket_soup.find_all('td',class_="table-body__cell name")
    Teams=Cricket_soup.find_all('td',class_="table-body__cell nationality-logo")
    Ratings=Cricket_soup.find_all('td',class_="table-body__cell u-text-right rating")
    
    for lIndex in range(9):
        Names_list.append(Names[lIndex].get_text().replace("\n",""))
        Teams_list.append(Teams[lIndex].get_text().replace("\n",""))
        Ratings_list.append(Ratings[lIndex].get_text().replace("\n",""))

    print(Names_list)
    print(Teams_list)
    print(Ratings_list)
    
    # Storing lists as columns/features in output dataframe
    Output_df['Names']=Names_list
    Output_df['Teams']=Teams_list
    Output_df['Ratings']=Ratings_list
    
    return(Output_df)

In [71]:
def fnGetTop10_ODIBowlers_Details(sURL):
    # DataFrame Initialization
    Output_df=pd.DataFrame()
    
    #List initializations
    Names_list=[]
    Teams_list=[]
    Ratings_list=[]
    
    # Requesting webpage for information from input URL
    Cricket_page=requests.get(sURL)
    
    # Reading the content
    Cricket_soup=BeautifulSoup(Cricket_page.content,'html.parser')
    
    # Getting information for the batsmen at first rank and storing them in respective lists
    Names=Cricket_soup.find_all('div',class_="rankings-block__banner--name")
    Teams=Cricket_soup.find_all('div',class_="rankings-block__banner--nationality")
    Ratings=Cricket_soup.find_all('div',class_="rankings-block__banner--rating")
    
    Names_list.append(Names[1].get_text().replace("\n",""))
    Teams_list.append(Teams[1].get_text().replace("\n",""))
    Ratings_list.append(Ratings[1].get_text().replace("\n",""))
    
    # Getting list of other 9 players
    Names=Cricket_soup.find_all('td',class_="table-body__cell name")
    Teams=Cricket_soup.find_all('td',class_="table-body__cell nationality-logo")
    Ratings=Cricket_soup.find_all('td',class_="table-body__cell u-text-right rating")
    
    for lIndex in range(9,18):
        Names_list.append(Names[lIndex].get_text().replace("\n",""))
        Teams_list.append(Teams[lIndex].get_text().replace("\n",""))
        Ratings_list.append(Ratings[lIndex].get_text().replace("\n",""))
    
    # Storing lists as columns/features in output dataframe
    Output_df['Names']=Names_list
    Output_df['Teams']=Teams_list
    Output_df['Ratings']=Ratings_list
    
    return(Output_df)

In [93]:
# Getting top 10 ODI Batsmen
sURL="https://www.icc-cricket.com/rankings/mens/player-rankings/odi"
BestODI_df=fnGetTop10_ODIBatsmen_Details(sURL)
BestODI_df

['Virat Kohli', 'Rohit Sharma', 'Babar Azam', 'Ross Taylor', 'Aaron Finch', 'Francois du Plessis', 'David Warner', 'Kane Williamson', 'Quinton de Kock', 'Jonny Bairstow']
['IND', 'IND', 'PAK', 'NZ', 'AUS', 'SA', 'AUS', 'NZ', 'SA', 'ENG']
['870', '842', '837', '818', '791', '790', '773', '765', '755', '754']


Names Teams Ratings
0          Virat Kohli   IND     870
1         Rohit Sharma   IND     842
2           Babar Azam   PAK     837
3          Ross Taylor    NZ     818
4          Aaron Finch   AUS     791
5  Francois du Plessis    SA     790
6         David Warner   AUS     773
7      Kane Williamson    NZ     765
8      Quinton de Kock    SA     755
9       Jonny Bairstow   ENG     754

In [72]:
# Getting top 10 ODI Bowlers
sURL="https://www.icc-cricket.com/rankings/mens/player-rankings/odi"
BestODI_df=fnGetTop10_ODIBowlers_Details(sURL)
BestODI_df

['Trent Boult', 'Mujeeb Ur Rahman', 'Jasprit Bumrah', 'Chris Woakes', 'Kagiso Rabada', 'Josh Hazlewood', 'Mohammad Amir', 'Pat Cummins', 'Matt Henry', 'Jofra Archer']
['NZ', 'AFG', 'IND', 'ENG', 'SA', 'AUS', 'PAK', 'AUS', 'NZ', 'ENG']
['722', '701', '700', '675', '665', '660', '647', '646', '641', '637']


Names Teams Ratings
0       Trent Boult    NZ     722
1  Mujeeb Ur Rahman   AFG     701
2    Jasprit Bumrah   IND     700
3      Chris Woakes   ENG     675
4     Kagiso Rabada    SA     665
5    Josh Hazlewood   AUS     660
6     Mohammad Amir   PAK     647
7       Pat Cummins   AUS     646
8        Matt Henry    NZ     641
9      Jofra Archer   ENG     637

## Q6. Getting details of Women cricket in ODI Cricket 

In [76]:
# Getting top 10 ODI team for women in ODI cricket
sURL="https://www.icc-cricket.com/rankings/womens/team-rankings/odi"
TopODITeams_df=fnGetTop10_Cricket_TeamDetails(sURL,'span',"u-hide-phablet",'td',
                "rankings-block__banner--matches",'td',"table-body__cell u-center-text",'td',
                "rankings-block__banner--points",'td',"table-body__cell u-center-text",'td',
                "rankings-block__banner--rating u-text-right",'td',"table-body__cell u-text-right rating")
TopODITeams_df

['Australia', 'India', 'England', 'South Africa', 'New Zealand', 'West Indies', 'Pakistan', 'Bangladesh', 'Sri Lanka', 'Ireland']
['15', '15', '14', '16', '15', '12', '12', '5', '11', '2']
['2,436', '1,812', '1,670', '1,713', '1,384', '1,025', '927', '306', '519', '25']
['162', '121', '119', '107', '92', '85', '77', '61', '47', '13']


Teams Total_Matches Points Ratings
0     Australia            15  2,436     162
1         India            15  1,812     121
2       England            14  1,670     119
3  South Africa            16  1,713     107
4   New Zealand            15  1,384      92
5   West Indies            12  1,025      85
6      Pakistan            12    927      77
7    Bangladesh             5    306      61
8     Sri Lanka            11    519      47
9       Ireland             2     25      13

In [79]:
# Top 10 ODI Batsmen in Women's ODI Cricket
sURL="https://www.icc-cricket.com/rankings/womens/player-rankings/odi"
ODIWomenRanking_df=fnGetTop10_ODIBatsmen_Details(sURL)
ODIWomenRanking_df

['Meg Lanning', 'Stafanie Taylor', 'Alyssa Healy', 'Smriti Mandhana', 'Amy Satterthwaite', 'Tammy Beaumont', 'Ellyse Perry', 'Lizelle Lee', 'Laura Wolvaardt', 'Mithali Raj']
['AUS', 'WI', 'AUS', 'IND', 'NZ', 'ENG', 'AUS', 'SA', 'SA', 'IND']
['749', '746', '741', '732', '723', '716', '691', '690', '689', '687']


Names Teams Ratings
0        Meg Lanning   AUS     749
1    Stafanie Taylor    WI     746
2       Alyssa Healy   AUS     741
3    Smriti Mandhana   IND     732
4  Amy Satterthwaite    NZ     723
5     Tammy Beaumont   ENG     716
6       Ellyse Perry   AUS     691
7        Lizelle Lee    SA     690
8    Laura Wolvaardt    SA     689
9        Mithali Raj   IND     687

In [94]:
def fnGetTop10_ODIWomen_Allrounders_Details(sURL):
    # DataFrame Initialization
    Output_df=pd.DataFrame()
    
    #List initializations
    Names_list=[]
    Teams_list=[]
    Ratings_list=[]
    
    # Requesting webpage for information from input URL
    Cricket_page=requests.get(sURL)
    
    # Reading the content
    Cricket_soup=BeautifulSoup(Cricket_page.content,'html.parser')
    
    # Getting information for the batsmen at first rank and storing them in respective lists
    Names=Cricket_soup.find_all('div',class_="rankings-block__banner--name")
    Teams=Cricket_soup.find_all('div',class_="rankings-block__banner--nationality")
    Ratings=Cricket_soup.find_all('div',class_="rankings-block__banner--rating")
    
    Names_list.append(Names[2].get_text().replace("\n",""))
    Teams_list.append(Teams[2].get_text().replace("\n",""))
    Ratings_list.append(Ratings[2].get_text().replace("\n",""))
    
    # Getting list of other 9 players
    Names=Cricket_soup.find_all('td',class_="table-body__cell name")
    Teams=Cricket_soup.find_all('td',class_="table-body__cell nationality-logo")
    Ratings=Cricket_soup.find_all('td',class_="table-body__cell u-text-right rating")
    
    for lIndex in range(18,27):
        Names_list.append(Names[lIndex].get_text().replace("\n",""))
        Teams_list.append(Teams[lIndex].get_text().replace("\n",""))
        Ratings_list.append(Ratings[lIndex].get_text().replace("\n",""))

    # Storing lists as columns/features in output dataframe
    Output_df['Names']=Names_list
    Output_df['Teams']=Teams_list
    Output_df['Ratings']=Ratings_list
    
    return(Output_df)

In [95]:
# Getting top 10 all rounders in ODI Women's cricket
sURL="https://www.icc-cricket.com/rankings/womens/player-rankings/odi"
ODIWomenAllRounder_df=fnGetTop10_ODIWomen_Allrounders_Details(sURL)
ODIWomenAllRounder_df

['Ellyse Perry', 'Stafanie Taylor', 'Marizanne Kapp', 'Deepti Sharma', 'Dane van Niekerk', 'Jess Jonassen', 'Sophie Devine', 'Natalie Sciver', 'Shikha Pandey', 'Katherine Brunt']
['AUS', 'WI', 'SA', 'IND', 'SA', 'AUS', 'NZ', 'ENG', 'IND', 'ENG']
['460', '410', '389', '359', '335', '301', '289', '273', '250', '232']


Names Teams Ratings
0      Ellyse Perry   AUS     460
1   Stafanie Taylor    WI     410
2    Marizanne Kapp    SA     389
3     Deepti Sharma   IND     359
4  Dane van Niekerk    SA     335
5     Jess Jonassen   AUS     301
6     Sophie Devine    NZ     289
7    Natalie Sciver   ENG     273
8     Shikha Pandey   IND     250
9   Katherine Brunt   ENG     232

## Q7. Get Phone details from amazon website with price less than 20000

In [210]:
# This function returns a dataframe comtaining phone details from the given URL
def fnGet_PhoneDetails_FromAmazon(sURL):
    
    Output_df=pd.DataFrame()
    
    # request url to access the information
    Mobiles=requests.get(sURL)
    
    # Access the data
    Mobiles_soup=BeautifulSoup(Mobiles.content,"html.parser")
    
    # List initializations
    ProdName_list=[]
    Price_list=[]
    ImageURL_list=[]
    Rating_list=[]
    lIndex_par=0
    
    # Collecting data from page for Prod Name, prices, Image url and average rating
    Products=Mobiles_soup.find_all('a',class_="a-link-normal a-text-normal")
    Prices=Mobiles_soup.find_all('span',class_="a-price-whole")
    Images=Mobiles_soup.find_all('img',class_="s-image")
    Ratings=Mobiles_soup.find_all('span',class_="a-icon-alt")
    
    lLen=min(len(Products),len(Prices),len(Images),len(Ratings))
    
    for lIndex in range(lLen):
        ImageURL_list.append(Images[lIndex]['src'].replace("\n",""))
        #Getting product name after removing details from name
        sText=Products[lIndex].get_text().replace("\n","")
        ProdName_list.append(sText[:sText.index("(")-1])
        
        Price_list.append(Prices[lIndex].get_text().replace("\n",""))
        # Getting ratings
        sText=Ratings[lIndex].get_text().replace("\n","")
        lIndex_par=sText.index(" ")
        Rating_list.append(sText[:lIndex_par])

    Output_df['Prod_Name']=ProdName_list
    Output_df['Price']=Price_list
    Output_df['Image_URL']=ImageURL_list
    Output_df['Rating']=Rating_list
    return(Output_df)

In [211]:
sURL="https://www.amazon.in/s?k=Mobiles&i=electronics&rh=n%3A976419031%2Cn%3A1389401031%2Cn%3A1389432031%2Cp_36%3A-2000000&dc&qid=1610275177&rnid=1318502031&ref=sr_nr_p_36_5"
    
# Access the data
Output_df=fnGet_PhoneDetails_FromAmazon(sURL)
Output_df

Prod_Name   Price  \
0        Samsung Galaxy M31  16,499   
1             Redmi 8A Dual   7,999   
2   Samsung Galaxy M01 Core   5,999   
3        Samsung Galaxy M21  13,999   
4                   Redmi 9   8,999   
5              Redmi Note 9  11,999   
6             Redmi 8A Dual   7,999   
7                  Redmi 9A   7,499   
8   Samsung Galaxy M01 Core   5,999   
9          Redmi Note 9 Pro  12,999   
10       Panasonic Eluga i7   5,000   
11       Samsung Galaxy M21  13,999   
12      Samsung Galaxy M31s  19,499   
13       Samsung Galaxy M31  16,499   
14             Redmi Note 8  11,499   
15                 Vivo U10   9,990   
16                 OPPO A5S   8,990   
17                 Redmi 9A   6,999   
18   I KALL K210 Smartphone   4,299   
19                 Oppo A52  13,990   
20  Samsung Galaxy M01 Core   4,999   
21  Samsung Galaxy M01 Core   4,999   
22                 Vivo Y50  16,490   
23                Vivo Y91i   7,990   

                                            Image_URL Rating  
0   https://m.media-amazon.com/images/I/71-Su4Wr0H...      4  
1   https://m.media-amazon.com/images/I/71yXShgxvp...      3  
2   https://m.media-amazon.com/images/I/71AYb2AGHX...      2  
3   https://m.media-amazon.com/images/I/71dujTTJDZ...      1  
4   https://m.media-amazon.com/images/I/71uZrDPrsR...    4.2  
5   https://m.media-amazon.com/images/I/81mtDLql+X...    4.0  
6   https://m.media-amazon.com/images/I/710p2f-zAd...    3.4  
7   https://m.media-amazon.com/images/I/712Ex7xDnd...    4.2  
8   https://m.media-amazon.com/images/I/71ahT-X2Fg...    4.0  
9   https://m.media-amazon.com/images/I/91Lr-OxVKj...    4.2  
10  https://m.media-amazon.com/images/I/41QsvcpKaZ...    4.0  
11  https://m.media-amazon.com/images/I/71QLvGIAq5...    4.1  
12  https://m.media-amazon.com/images/I/61d-phh4Gf...    3.4  
13  https://m.media-amazon.com/images/I/71OxJeyywS...    4.2  
14  https://m.media-amazon.com/images/I/61wXFmyMLQ...    2.9  
15  https://m.media-amazon.com/images/I/51EnyMldjG...    4.2  
16  https://m.media-amazon.com/images/I/71ZU4Vsevq...    4.3  
17  https://m.media-amazon.com/images/I/71sxX9aLtn...    4.2  
18  https://m.media-amazon.com/images/I/61R-EX3JF7...    4.3  
19  https://m.media-amazon.com/images/I/71FPmXaDfb...    4.3  
20  https://m.media-amazon.com/images/I/71AYb2AGHX...    4.1  
21  https://m.media-amazon.com/images/I/71ahT-X2Fg...    4.1  
22  https://m.media-amazon.com/images/I/514ETCdN3C...    4.1  
23  https://m.media-amazon.com/images/I/51FnmxwFHD...    3.4

## Q8. Get weather report from weather.com 

In [271]:
def fnGet_WeatherReportFor_NextSevenDays(sURL):
    
    Output_df=pd.DataFrame()
    
    # request url to access the information
    Weathers=requests.get(sURL)
    
    # Access the data
    Weathers_soup=BeautifulSoup(Weathers.content,"html.parser")
    
    # List initializations
    Period_list=[]
    ShortDesc_list=[]
    Temp_list=[]
    Description_list=[]
    
    # Collecting data from page for Prod Name, prices, Image url and average rating
    Period=Weathers_soup.find_all('p',class_="period-name")
    Short_Desc=Weathers_soup.find_all('p',class_="short-desc")
    high_Temp=Weathers_soup.find_all('p',class_="temp temp-high")
    low_Temp=Weathers_soup.find_all('p',class_="temp temp-low")
    Description=Weathers_soup.find_all('div',class_="col-sm-10 forecast-text")
    
    lLen=min(len(Period),len(Short_Desc),len(high_Temp),len(low_Temp),len(Description))
    l=0
    k=0
    for lIndex in range(2*lLen):
        Period_list.append(Period[lIndex].get_text().replace("\n",""))
        ShortDesc_list.append(Short_Desc[lIndex].get_text().replace("\n",""))
        Description_list.append(Description[lIndex].get_text().replace("\n",""))
        if lIndex%2==0: 
            Temp_list.append(high_Temp[l].get_text().replace("\n",""))
            l+=1
        else:
            Temp_list.append(low_Temp[k].get_text().replace("\n",""))
            k+=1
        
    Output_df['Period']=Period_list
    Output_df['Shrot_Desc']=ShortDesc_list
    Output_df['Temperature']=Temp_list
    Output_df['Description']=Description_list
    return(Output_df)

In [272]:
sURL=("https://forecast.weather.gov/MapClick.php?lat=37.7772&lon=-122.4168#.X_xCOpMzbep")
Weather_df=fnGet_WeatherReportFor_NextSevenDays(sURL)
Weather_df

Period                  Shrot_Desc  Temperature  \
0          Today        Patchy Fogthen Sunny  High: 65 °F   
1        Tonight  Mostly Clearthen PatchyFog   Low: 48 °F   
2         Friday                Partly Sunny  High: 64 °F   
3    FridayNight               Partly Cloudy   Low: 48 °F   
4       Saturday                Mostly Sunny  High: 63 °F   
5  SaturdayNight                Mostly Clear   Low: 48 °F   
6         Sunday                       Sunny  High: 65 °F   
7    SundayNight                Mostly Clear   Low: 51 °F   

                                         Description  
0  Patchy fog before 10am.  Otherwise, mostly sun...  
1  Patchy fog between 4am and 5am.  Otherwise, pa...  
2  Partly sunny, with a high near 64. Calm wind b...  
3  Partly cloudy, with a low around 48. West wind...  
4  Mostly sunny, with a high near 63. Calm wind b...  
5                Mostly clear, with a low around 48.  
6                        Sunny, with a high near 65.  
7                Mostly clear, with a low around 51.

## Q9-10. Extracting data from Monster.com

In [2]:
import selenium
import pandas as pd
from selenium import webdriver

In [3]:
def fnReturnsDF_WithJobs_ListedOnMonster(sURL):
    driver=webdriver.Chrome(r"C:\Users\Sankalp\chromedriver_win32 (2)\chromedriver.exe")
    driver.get(sURL)
    
    #Initializations
    Jobs_list=[]
    Companies_list=[]
    Locations_list=[]
    Experience_list=[]
    Output_df=pd.DataFrame()
    
    # Scraping data from the website, given their respective tags
    Job_titles=driver.find_elements_by_xpath("//h3[@class='medium']")
    Companies=driver.find_elements_by_xpath("//span[@class='company-name']")
    Locations=driver.find_elements_by_xpath("//div[@class='col-xxs-12 col-sm-5 text-ellipsis']")
    Experience=driver.find_elements_by_xpath("//div[@class='exp col-xxs-12 col-sm-3 text-ellipsis']")
    print(len(Job_titles),len(Companies),len(Locations),len(Experience))
    for lIndex in range(len(Job_titles)):
        Jobs_list.append(Job_titles[lIndex].text)
        Companies_list.append(Companies[lIndex].text)
        Locations_list.append(Locations[lIndex].text)
        Experience_list.append(Experience[lIndex].text)    
   
    Output_df['Jobs']=Jobs_list
    Output_df['Companies']=Companies_list
    Output_df['Locations']=Locations_list
    Output_df['Experience']=Experience_list
    
    return(Output_df)

In [4]:
################### Extracting Software Developers job from given URLs #############################
# Final empty dataframe
Jobs_df=pd.DataFrame()
# Frist page URL
sFirstPageURL="https://www.monsterindia.com/srp/results?query=software%20developer&searchId=fec59775-b562-4631-869a-2145c2a15a34"
Output_df=fnReturnsDF_WithJobs_ListedOnMonster(sFirstPageURL)
Jobs_df=Output_df.copy()
# Looping over next 4 pages and collecting the data
for ObservedInstances in range(25,101,25):
    sDynamicPageURLS="https://www.monsterindia.com/srp/results?start=" + str(ObservedInstances) + "&sort=1&limit=25&query=software%20developer&searchId=fec59775-b562-4631-869a-2145c2a15a34"
    # Calling function to extract data for software developers
    Output_df=fnReturnsDF_WithJobs_ListedOnMonster(sDynamicPageURLS)
    # Concatinating the new and the old concatenated DataFrames
    Jobs_df=pd.concat([Jobs_df,Output_df],ignore_index=True,sort=False)

print(Jobs_df.shape)
Jobs_df.head(20)

25 25 25 25
25 25 25 25
25 25 25 25
25 25 25 25
25 25 25 25
(125, 4)


Jobs  \
0                         Software Developer   
1                      Sr. Software Engineer   
2   Hiring for Java ,Microservices, Big data   
3                   senior software Engineer   
4                          Software Engineer   
5                             Cyber Security   
6                       Mean Stack Developer   
7                           Mobile Developer   
8                            System Security   
9                          Computer Operator   
10            Core Java Developer_Jio_Mumbai   
11                              video editor   
12      Nagios administrator- work from home   
13                          Software Testing   
14                       Computer Networking   
15                  Desktop Support Engineer   
16                              Data Analyst   
17                          Python Developer   
18                             Php developer   
19                             PHP Developer   

                                            Companies  \
0                           Company Name Confidential   
1                Deputize Consultancy Private limited   
2          Anri Solutions HR Services Private Limited   
3    Yashodha S M (Proprietor Of Co Inter Technology)   
4    Yashodha S M (Proprietor Of Co Inter Technology)   
5    Yashodha S M (Proprietor Of Co Inter Technology)   
6   Delivery Centric Technology Services Private L...   
7    Yashodha S M (Proprietor Of Co Inter Technology)   
8    Yashodha S M (Proprietor Of Co Inter Technology)   
9    Yashodha S M (Proprietor Of Co Inter Technology)   
10  Quess IT Staffing (Formerly known as Magna Inf...   
11   Yashodha S M (Proprietor Of Co Inter Technology)   
12                     Millennium Ims Private Limited   
13   Yashodha S M (Proprietor Of Co Inter Technology)   
14   Yashodha S M (Proprietor Of Co Inter Technology)   
15   Yashodha S M (Proprietor Of Co Inter Technology)   
16   Yashodha S M (Proprietor Of Co Inter Technology)   
17  Hiren B Patel (Proprietor Of Bajaj India Service)   
18               Deputize Consultancy Private limited   
19                          Company Name Confidential   

                     Locations  Experience  
0     Hyderabad / Secunderabad   0-1 Years  
1                      Lucknow   5-8 Years  
2                      Chennai   6-8 Years  
3        Bengaluru / Bangalore   0-2 Years  
4                   Chandrapur   0-2 Years  
5                    Fatehabad   0-2 Years  
6        Bengaluru / Bangalore   3-5 Years  
7   Cochin / Kochi / Ernakulam   0-2 Years  
8                  Bhubaneswar   0-2 Years  
9                      Cuttack   0-2 Years  
10                      Mumbai   2-6 Years  
11                      Mumbai   0-2 Years  
12       Bengaluru / Bangalore  8-13 Years  
13                    Guwahati   0-2 Years  
14                       Erode   0-2 Years  
15                      Mumbai   0-2 Years  
16    Hyderabad / Secunderabad   0-2 Years  
17                   Faridabad   0-2 Years  
18                     Lucknow   3-5 Years  
19                      Indore   1-5 Years

In [5]:
################### Extracting Data Science jobs in Delhi from given URLs #############################
# Looping over next 4 pages and collecting the data
for ObservedInstances in range(25,151,25):
    sDynamicPageURLS="https://www.monsterindia.com/srp/results?start=" + str(ObservedInstances) + "&sort=1&limit=25&query=Data%20Science&locations=Delhi&searchId=2a3eb8d6-d687-4f58-987a-72bf438131ba"
    # Calling function to extract data for software developers
    Output_df=fnReturnsDF_WithJobs_ListedOnMonster(sDynamicPageURLS)
    # Concatinating the new and the old concatenated DataFrames
    Jobs_df=pd.concat([Jobs_df,Output_df],ignore_index=True,sort=False)
print(Jobs_df.shape)
Jobs_df.head(20)

25 25 25 25
25 25 25 25
25 25 25 25
25 25 25 25
25 25 25 25
25 25 25 25
(275, 4)


Jobs  \
0                         Software Developer   
1                      Sr. Software Engineer   
2   Hiring for Java ,Microservices, Big data   
3                   senior software Engineer   
4                          Software Engineer   
5                             Cyber Security   
6                       Mean Stack Developer   
7                           Mobile Developer   
8                            System Security   
9                          Computer Operator   
10            Core Java Developer_Jio_Mumbai   
11                              video editor   
12      Nagios administrator- work from home   
13                          Software Testing   
14                       Computer Networking   
15                  Desktop Support Engineer   
16                              Data Analyst   
17                          Python Developer   
18                             Php developer   
19                             PHP Developer   

                                            Companies  \
0                           Company Name Confidential   
1                Deputize Consultancy Private limited   
2          Anri Solutions HR Services Private Limited   
3    Yashodha S M (Proprietor Of Co Inter Technology)   
4    Yashodha S M (Proprietor Of Co Inter Technology)   
5    Yashodha S M (Proprietor Of Co Inter Technology)   
6   Delivery Centric Technology Services Private L...   
7    Yashodha S M (Proprietor Of Co Inter Technology)   
8    Yashodha S M (Proprietor Of Co Inter Technology)   
9    Yashodha S M (Proprietor Of Co Inter Technology)   
10  Quess IT Staffing (Formerly known as Magna Inf...   
11   Yashodha S M (Proprietor Of Co Inter Technology)   
12                     Millennium Ims Private Limited   
13   Yashodha S M (Proprietor Of Co Inter Technology)   
14   Yashodha S M (Proprietor Of Co Inter Technology)   
15   Yashodha S M (Proprietor Of Co Inter Technology)   
16   Yashodha S M (Proprietor Of Co Inter Technology)   
17  Hiren B Patel (Proprietor Of Bajaj India Service)   
18               Deputize Consultancy Private limited   
19                          Company Name Confidential   

                     Locations  Experience  
0     Hyderabad / Secunderabad   0-1 Years  
1                      Lucknow   5-8 Years  
2                      Chennai   6-8 Years  
3        Bengaluru / Bangalore   0-2 Years  
4                   Chandrapur   0-2 Years  
5                    Fatehabad   0-2 Years  
6        Bengaluru / Bangalore   3-5 Years  
7   Cochin / Kochi / Ernakulam   0-2 Years  
8                  Bhubaneswar   0-2 Years  
9                      Cuttack   0-2 Years  
10                      Mumbai   2-6 Years  
11                      Mumbai   0-2 Years  
12       Bengaluru / Bangalore  8-13 Years  
13                    Guwahati   0-2 Years  
14                       Erode   0-2 Years  
15                      Mumbai   0-2 Years  
16    Hyderabad / Secunderabad   0-2 Years  
17                   Faridabad   0-2 Years  
18                     Lucknow   3-5 Years  
19                      Indore   1-5 Years